# Example: Net Present Value for a Tesla Model S
In this example, we'll compute the [Net Present Value (NPV)](https://en.wikipedia.org/wiki/Net_present_value) of purchasing, owning, and selling a Tesla Model S. Using the Abstract Asset approach, we'll model the cash flow events throughout the vehicle's ownership lifecycle to determine if this represents a sound investment.

### Learning objectives
This example will familiarize students with computing the [Net Present Value (NPV)](https://en.wikipedia.org/wiki/Net_present_value), i.e., specifying the net cash flow events, computing the discount factors for different periods and ultimately computing the net present value.
* __Prerequisite__: Setup problem components and constants. Before we do the [Net Present Value (NPV) calculation](https://en.wikipedia.org/wiki/Net_present_value#:~:text=NPV%20is%20the%20sum%20of,NPV%20results%20in%20a%20loss.), we'll set up some components and constants that we'll use later.
* __Task 1__: Setup the Cash Flow Dictionary. In this task, we specify some values for the cash flow events we observe over the lifetime of owning the [Tesla Model S](https://www.tesla.com/models). Then, we construct a net cash flow dictionary that holds the net values for the cash flow events during each period.
* __Task 2__: In this task, we compute a dictionary of discount factor $\mathcal{D}_{j,0}(\bar{r})$ values using the time and discount rate data specified in the Prerequisites section.
* __Task 3__: In this task, we'll compute the net present value (NPV) for the [Tesla Model S](https://www.tesla.com/models) and answer some discussion questions about how the value may change if we change our assumptions and how we should interpret the values produced by the calculation.

Let's go!
___

## Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 

> The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Otherwise, packages are downloaded and then loaded.

In [67]:
include(joinpath(@__DIR__, "Include.jl")); # this sets up the environment, we'll do this all the time, on everything we do

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

###  Problem components and constants
Before we do the [Net Present Value (NPV) calculation](https://en.wikipedia.org/wiki/Net_present_value#:~:text=NPV%20is%20the%20sum%20of,NPV%20results%20in%20a%20loss.), we'll set up some components and constants that we'll use later. Let's start with the discounting model. 

For simplicity, we'll assume discrete compounding in this calculation. We specify this by creating an instance of [the `DiscreteCompoundingModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.DiscreteCompoundingModel) and save it in the `discount_model::DiscreteCompoundingModel` variable.

In [70]:
discount_model = DiscreteCompoundingModel();

Next, we set several constants, which we use below. For a description of the constant and permissible values, see the comment string beside the declaration.

In [72]:
T = 10.0;  # number of years; value ≥ 0
λ = 2.0 |> Int;  # number of periods per year (λ = 2.0 => every six months); value ≥ 1
r̄ = 0.0425; # TODO: discount rate per year; value ≥ 0
depreciation = 0.0625; # TODO: price decline per period (semiannual)
N = λ*T |> Int;  # total number of observations

## Task 1: Setup the Cash Flow Dictionary
In this task, we specify values for the cash flow events throughout the Tesla Model S ownership period. Then, we construct a net cash flow dictionary that holds the net values for each period. Let's start with the key parameters.

In [74]:
purchase_price = 111630; # how much do we pay now for the Model S?
                         # this price is added to all the cool options kachowwwww!!
sale_price = max(0.0, (1 - N*depreciation)*purchase_price); # assumes straight line depreciation
insurance_costs = 1808.0 # 1808.0 How much does a Model S cost to insure (per period)?
other_costs = 50.0; # Other costs associated with the Model S per period
other_savings = 0.0; # Other savings per period?

In [75]:
sale_price

0.0

Now, let's build and populate the __cash flow event dictionary__. The dictionary will hold the net cash flow events for each $N = \lambda\cdot{T}$ period. 

We'll populate this dictionary by iterating over the `0...N` periods using a [Julia `for-loop`](https://docs.julialang.org/en/v1/base/base/#for).
* For each period (index `i`), we check if we are `i == 0` (purchase), or `i == N` (sale) of the car. At purchase time, we have a negative cash flow (we pay the purchase price to Tesla), and at sale time we receive the sale price plus any savings minus expenses for that period.
* If $i\neq{0}\,||\,i\neq{N}$: we have an intermediate period where we make insurance payments net of any other savings occurring during that period.

The keys of the `cash_flow_event_dictionary` are the period indexes `i`, while the values are the net cash flow events for that period.

In [109]:
cash_flow_event_dictionary = let

    # initialize
    cash_flow_event_dictionary = Dict{Int64,Float64}();
    
    # main loop -
    for i ∈ 0:N
        if (i == 0)
        cash_flow_event_dictionary[i] = -1*purchase_price;
        elseif (i == N)
            cash_flow_event_dictionary[i] = sale_price + other_savings - (insurance_costs+other_costs)
        else
        cash_flow_event_dictionary[i] = other_savings - (insurance_costs+other_costs);
        end
    end

    cash_flow_event_dictionary # return
end;

In [111]:
# ps. this one includes both present value and future value
cash_flow_event_dictionary

Dict{Int64, Float64} with 21 entries:
  5  => -1858.0
  16 => -1858.0
  20 => -1858.0
  12 => -1858.0
  8  => -1858.0
  17 => -1858.0
  1  => -1858.0
  19 => -1858.0
  0  => -111630.0
  6  => -1858.0
  11 => -1858.0
  9  => -1858.0
  14 => -1858.0
  3  => -1858.0
  7  => -1858.0
  4  => -1858.0
  13 => -1858.0
  15 => -1858.0
  2  => -1858.0
  10 => -1858.0
  18 => -1858.0

## Task 2: Compute the Discount Factor Dictionary
In this task, we compute discount factors $\mathcal{D}_{j,0}(\bar{r})$ for each time period using the specified discount rate. We call [the `discount(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.discount-Tuple%7BAbstractCompoundingModel,%20Float64,%20Int64%7D) which returns these factors in a dictionary.

In [80]:
discount_dictionary = discount(discount_model, r̄, N, λ = λ);

`Unhide` the code block below to see how we built a table of the data in the `discount_dictionary` using [the `pretty_table(...)` function exported by the PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl) and [a `DataFrame` instance exported by the DataFrames.jl package](https://github.com/JuliaData/DataFrames.jl)

In [82]:
let
    # initialize -
    df = DataFrame();
    for i ∈ 0:N
        value = discount_dictionary[i];
        row_df = (
            period = i,
            𝒟 = value,
            𝒟inv = (1/value)
        );
        push!(df, row_df);
    end

    pretty_table(df, tf = tf_simple)
end

========= ========= ===========
  period         𝒟       𝒟inv 
   Int64   Float64    Float64 
========= ========= ===========
       0       1.0        1.0
       1   1.02125   0.979192
       2   1.04295   0.958817
       3   1.06511   0.938866
       4   1.08775   0.919331
       5   1.11086   0.900201
       6   1.13447    0.88147
       7   1.15858   0.863129
       8    1.1832   0.845169
       9   1.20834   0.827583
      10   1.23402   0.810362
      11   1.26024   0.793501
      12   1.28702    0.77699
      13   1.31437   0.760822
      14    1.3423   0.744991
      15   1.37082   0.729489
      16   1.39995    0.71431
      17    1.4297   0.699447
      18   1.46008   0.684893
      19   1.49111   0.670642
      20   1.52279   0.656687
========= ========= ===========


In [ ]:
# that means the present, you can pay $1 to get $1,
# but in the future, $1 from the present values only $0.656 in the future


### Check: Do we recover the discount rate $\bar{r}$?
Let's do a quick check on the discount factor calculation. If the discount factors are correct, we should be able to recover the discount rate $\bar{r}$ by inverting the definition of the discount factor, i.e., the discount rate should be equal to:
$$
\begin{equation}
\bar{r} = \lambda\cdot\left(\mathcal{D}_{i,0}^{1/i} - 1\right)
\end{equation}
$$
where $\mathcal{D}_{i,0}$ is the discount factor, $i$ is the period-index, and $\lambda$ is the number of compounding events per-period. This is the value for the period $i\geq{1}$. 

> __Test:__ Iterate over the `discount_dictionary,` and for iteration $i>0$, compute the discount rate (stored in the `r̄ᵢ` variable). Compare the estimated value  the specified (correct) discount rate `r̄.` We use [the Julia `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `isapprox(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to check for equality. 

If the check fails, [an AssertionError is thrown](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) and the loop terminates. So what happens?

In [113]:
let
    r̄ᵢ = r̄;
    for (k,v) ∈ discount_dictionary
        if (k > 0)
            r̄ᵢ = λ*((v)^(1/k) - 1);
        end
        @assert isapprox(r̄, r̄ᵢ, rtol = 1e-4); # check every period
    end
end

## Task 3: Compute the Net Present Value (NPV)
In this task, we'll compute the NPV for the Tesla Model S. The NPV is the dot product of the inverse discount factors and net cash flows:
$$
\begin{equation*}
\texttt{NPV} = \left<\mathcal{D}^{-1}\left(\bar{r}\right), \bar{c} \right>
\end{equation*}
$$

Since our data is stored in dictionaries, we'll convert them to arrays first, then use [the `dot(...)` method](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.dot) to compute the scalar product.

#### Compute the inverse discount factor array $\mathcal{D}^{-1}(\bar{r})$
To compute the inverse discount factor array $\mathcal{D}^{-1}(\bar{r})$, we iterate over the periods `i ∈ 1...N+1` (where we convert to `1`-based indexing). We insert the inverse of the value in the `discount_dictionary` for each period `i` into the `𝒟inv::Array{Float64,1}` array.

In [115]:
𝒟inv = let
    
    # initialize -
    𝒟inv = Array{Float64,1}(undef, N+1);
    
    # convert
    for i in 1:(N+1)
        j = i - 1;
        𝒟inv[i] = 1/discount_dictionary[j]
    end
    
    𝒟inv; # return
end;

#### Compute the cash flow event array $\bar{c}$.
To compute the cash flow event array $\bar{c}$, we iterate over the periods `i ∈ 1...N+1` (where we convert to `1`-based indexing). We insert the value from the `cash_flow_event_dictionary` for each period `i` into the `c̄::Array{Float64,1}` array.

In [117]:
c̄ = let

    # initialize -
    c̄ = Array{Float64,1}(undef, N+1);
    
    # convert -
    for i ∈ 1:(N+1)
        j = i - 1;
        c̄[i] = cash_flow_event_dictionary[j]
    end
    
    c̄; # return
end;

Finally, we compute the $\texttt{NPV}$ using [the `dot(...)` method exported by the LinearAlgebra.jl package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.dot).

In [119]:
NPV = dot(𝒟inv, c̄);
println("The NPV for a Tesla Model S over $(T) years equals $(NPV) USD")

The NPV for a Tesla Model S over 10.0 years equals -141647.6479753733 USD


In [ ]:
# it returns a minus price of the money |> it is not worth it to invest in Tesla Model S...

### Discussion Questions
* What does a negative NPV indicate about this investment?
* What factors could we change to improve the NPV of the Tesla Model S?

___

In [ ]:
# ex. other_savings: if we got paid $20,000 a year (other_savings = 10,000 because it times 2 units)
# then we will get 1584.7749...USD |> then it is good for the perspective for the investment!!

#...

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.